## 使用$(x,y)$坐标训练
坐标单位为m，速度单位为m/天
### 1：模型设置

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchdiffeq import odeint  # 神经ODE库

df = pd.read_csv('./xy_orbit.csv')
array = df.to_numpy()
array = array.T
t = array[0]/100
y = array[1:]/1e10

coords = torch.tensor(y.T, dtype=torch.float32)
t_tensor = torch.tensor(t,dtype=torch.float32)

print(coords.shape)

# 2. 定义ODE模型
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 32),
            nn.Tanh(),
            nn.Linear(32, 32)  
        )
        self.fc1 = nn.Linear(32,20)
        self.fc2 = nn.Linear(20,4)

    def forward(self, t, y):
        y = self.net(y)
        y= torch.sin(y)
        y = self.fc1(y)
        y = torch.tanh(y)
        y = self.fc2(y)

        return y

# 早停用函数
def errors(data):
    mean_value = np.mean(data)
    errors = data - mean_value
    absolute_errors = np.abs(errors)
    mae = np.mean(absolute_errors)
    return mae

'''# 初始化ODE函数和模型
model = ODEFunc()
optimizer = optim.Adam(model.parameters(), lr=0.01,weight_decay=1e-3)
criterion = nn.MSELoss()
epochs = 2000'''

# 超参数设置
loss_value = []
mini_delta = 0.005
sum_ = np.array([0,0,0,0,0])

# 定义训练函数
def train(model,optimizer,criterion,epochs):
    model.train()
    for epoch in range(epochs+1):
        optimizer.zero_grad()
        
        # 神经ODE求解
        pred_y = odeint(model, coords[0], t_tensor)
        
        # 计算损失
        loss = criterion(pred_y, coords)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        # 打印损失
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

        #设置早停
        loss_value.append(loss.item())
        if epoch > 5:
            for i in range(5):
                sum_[i] = loss_value[epoch]-loss_value[epoch-i-1]
        err = errors(sum_)

        if loss.item() < 0.001 and err < 0.0001 :
            break

### 2.模型训练

In [ ]:
model = ODEFunc()
model.load_state_dict(torch.load('xy_orbit_model_weights.pth'))
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-3)
criterion = nn.MSELoss()
epochs = 3000

train(model,optimizer,criterion,epochs)

In [ ]:
# 4. 可视化结果
model.eval()
time = torch.linspace(0,7,700*12)
with torch.no_grad():
    pred_y = odeint(model, coords[0], t_tensor)
    pred_y = pred_y.numpy()
    pred_y2 = odeint(model, coords[-1], time)
    pred_y2 = pred_y2.numpy()


# 绘制真实轨迹和预测轨迹
plt.figure(figsize=(6, 6))
#plt.plot(y[0],y[1] , 'b-', label="True Circle Path")
#plt.plot(pred_y[:, 0], pred_y[:, 1], 'r--', label="Predicted Path by ODE")
plt.plot(pred_y2[:, 0], pred_y2[:, 1], 'g--',linewidth = 0.5, label="futher_Predicted Path by ODE")
plt.xlabel('X')
plt.ylabel('Y')
plt.title('True Circle and Neural ODE Predicted Path')
plt.axis('equal')
plt.legend()
plt.show()

### 附录：训练效果记录
<div style="text-align: center;">
    <img src="output2.png" alt="1000步的训练" width="200"/>
    <p style="text-align: center;">1000步的训练，使用了4层线性层，初见效果</p>
</div>
<div style="text-align: center;">
    <img src="output3.png" alt="1000步的训练" width="200"/>
    <p style="text-align: center;">3000层的训练，使用了4层线性层，有一定的效果</p>
</div>
<div style="text-align: center;">
    <img src="output4.png" alt="1000步的训练" width="500"/>
    <p style="text-align: center;">6000层的训练，使用了4层线性层，有一定的效果:0.0012470250949263573</p>
</div>

## 函数的显式表达式：
函数的显示表达无法展开研究，过于大的数据量，只可嵌套变量去表出，无法观察到其函数模样，这也是为什么说是黑盒的原因吧

In [1]:
import torch

# 1. 加载模型参数字典
state_dict = torch.load('xy_orbit_model_weights.pth')

# 2. 查看字典内容分布
print("模型参数字典分布：")
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")  # 只显示每个参数的形状


模型参数字典分布：
net.0.weight: torch.Size([32, 4])
net.0.bias: torch.Size([32])
net.2.weight: torch.Size([32, 32])
net.2.bias: torch.Size([32])
fc1.weight: torch.Size([20, 32])
fc1.bias: torch.Size([20])
fc2.weight: torch.Size([4, 20])
fc2.bias: torch.Size([4])


C:\Users\19464\AppData\Local\Temp\ipykernel_6020\1745610918.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('xy_orbit_model_weights.pth')


In [1]:
import torch
import sympy as sp

# 1. 创建符号输入变量
n = 4  # 假设输入维度为4
x = sp.Matrix(sp.symbols(f'x:{n}'))  # 创建输入向量 x

# 2. 加载模型参数字典
state_dict = torch.load('xy_orbit_model_weights.pth')

# 提取每层的权重和偏置参数
# 假设模型有两层线性层和一个 Tanh 激活函数
W1 = sp.Matrix(state_dict['net.0.weight'].numpy())
b1 = sp.Matrix(state_dict['net.0.bias'].numpy())
W2 = sp.Matrix(state_dict['net.2.weight'].numpy())
b2 = sp.Matrix(state_dict['net.2.bias'].numpy())
W3 = sp.Matrix(state_dict['fc1.weight'].numpy())
b3 = sp.Matrix(state_dict['fc1.bias'].numpy())
W4 = sp.Matrix(state_dict['fc2.weight'].numpy())
b4 = sp.Matrix(state_dict['fc2.bias'].numpy())

# 3. 构建符号化表达式
# 第一层线性变换
y1 = W1 * x + b1

# Tanh 激活函数
z1 = y1.applyfunc(sp.tanh)  # 对每个元素应用 tanh 函数

# 第二层线性变换
y2 = W2 * z1 + b2
print("the function is: f2(x) =")
sp.pprint(y2)

'''# sin激活函数
z2 = y2.applyfunc(sp.sin)

# 第三层线性变换
y3 = W3 * z2 + b3

# Tanh 激活函数
z3 = y3.applyfunc(sp.tanh)  # 对每个元素应用 tanh 函数

# 第四层线性变换
y4 = W4 * z3 + b4

# 4. 输出显式表达式
print("the function is: f(x) =")
sp.pprint(y4)'''


C:\Users\19464\AppData\Local\Temp\ipykernel_2684\4230737085.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('xy_orbit_model_weights.pth')


the function is: f2(x) =
⎡           0.145605⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.04125
⎢                                                                             
⎢     0.310072⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.0412513⋅x₃ 
⎢                                                                             
⎢         -0.0848371⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.04125
⎢                                                                             
⎢       0.163578⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.0412513⋅x
⎢                                                                             
⎢     0.0758246⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.0412513⋅x₃
⎢                                                                             
⎢      -0.143906⋅tanh(-0.312447⋅x₀ + 0.528507⋅x₁ + 0.00637158⋅x₂ + 0.0412513⋅x
⎢                                                                             
⎢         0.374038⋅tanh(-0.

'# sin激活函数\nz2 = y2.applyfunc(sp.sin)\n\n# 第三层线性变换\ny3 = W3 * z2 + b3\n\n# Tanh 激活函数\nz3 = y3.applyfunc(sp.tanh)  # 对每个元素应用 tanh 函数\n\n# 第四层线性变换\ny4 = W4 * z3 + b4\n\n# 4. 输出显式表达式\nprint("the function is: f(x) =")\nsp.pprint(y4)'